In [2]:
import pandas as pd
import dash
from dash import dcc, html, Dash, dash_table, Input, Output
import csv
import folium
from map_routing import *

# Load the data frame from a CSV file, change it if in another region
df = pd.read_csv('intermediate_v2.csv')

# Init the map 
# and save it as html so we can embed in the dash app
init_map = folium.Map(location=[22.3193, 114.1694], zoom_start=12)
init_map.save('map.html','w')
graph = loadGraph("hongkong_speed.graphml")

# Create the Dash app
app = dash.Dash(__name__)

# BG color
colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

# Define the layout
app.layout = html.Div(
    children=[
       
       # 1.heading
        html.Div(style={'backgroundColor': colors['background']}, children=[
            # 1.1 Title
            html.H1(
                children='LI & FUNG SHIPPING DASHBOARD',
                style={
                    'textAlign': 'center',
                    'color': colors['text']
                }
            ),
            # 1.2 Subtitle
            html.Div(children='HKUST IEDA FYP GROUP 3', style={
                'textAlign': 'center',
                'color': colors['text']
            })
        ]),

        # Filter section, options on top, filtered results on bottom
        html.Div(children=[
            html.H4('Filtering'),
            # filter option
            html.Div(style={'border': '1px solid black', 'padding': '10px', 'margin-bottom': '10px'},
                     children=[
                         html.Div(children=[
                             html.H5('Location', style={'textAlign': 'center'}),
                             dcc.Dropdown(
                                 id='location-dropdown',
                                 options=[{'label': location, 'value': location} for location in df.Loc.unique()],
                                 value='All Locations',
                                 clearable=False
                             )
                         ]),
                         html.Div(children=[
                             html.H5('Warehouse', style={'textAlign': 'center'}),
                             dcc.Dropdown(
                                 id='warehouse-dropdown',
                                 options=[{'label': warehouse, 'value': warehouse} for warehouse in
                                          df.warehouse.unique()],
                                 value='All Warehouses',
                                 clearable=False
                             )
                         ]),
                         html.Div(children=[
                             html.H5('Van Type', style={'textAlign': 'center'}),
                             dcc.Dropdown(
                                 id='van-type-dropdown',
                                 options=[{'label': van_type, 'value': van_type} for van_type in
                                          df.Vehicle.unique()],
                                 value='All Van Types',
                                 clearable=False
                             )
                         ]),
                         html.Div(children=[
                             html.H5('Start Time', style={'textAlign': 'center'}),
                             dcc.Dropdown(
                                 id='start-time-dropdown',
                                 options=[{'label': start_time, 'value': start_time} for start_time in
                                          df.planStartTime.unique()],
                                 value='All Start Locations',
                                 clearable=False
                             )
                         ])
                     ]),
            
            #filter result
            html.Div(
                id='filtered-table-container',
                style={'margin-top': '20px', 'border': '1px solid black', 'padding': '10px', 'font-size': '14px'}
            ),
        ], style={'width': '40%', 'display': 'inline-block', 'vertical-align': 'top'}),
        
        # Map + Detail section, Map on top, Details on bottom
        html.Div(children=[
            html.H4('Map'),
            
            # Map ,srcDoc generated by folium_object.save().
            # call_back function on this to update real-time
            html.Iframe(srcDoc=open('map.html', 'r').read(),
                   style={'width': '100%', 'vertical-align': 'top', 'height': '58vh', 'margin': "auto",
                          "display": "block"},
                   id="map"),

            # shipment details, here just outlines the position
            # call_back function on this to update
            html.Div(id='shipment-details-container',
                    children=[
                        html.Div('Shipment Details', style={'font-size': '16px', 'font-weight': 'bold', 'text-align': 'left', 'margin-left': '1%', 'margin-top': '1%'}),
                        html.Div(
                            [html.Div(style={'border': '1px solid black', 'height': '200px', 'width': '200px', 'display': 'inline-block', 'margin-right': '40px'}),
                            html.Div(style={'border': '1px solid black', 'height': '200px', 'width': '200px', 'display': 'inline-block', 'margin-right': '40px'}),
                            html.Div(style={'border': '1px solid black', 'height': '200px', 'width': '200px','display': 'inline-block', 'margin-right': '40px'}),
                            html.Div(style={'border': '1px solid black', 'height': '200px', 'width': '200px','display': 'inline-block'}),                ],
                            style={'display': 'inline-block', 'margin-top': '10px', 'margin-left': '1%'}
                        ),
                    ],
                    style={
                        'border': '1px solid black',
                        'height': '250px',
                        'text-align': 'center',
                    },
                )

        ], style={'margin-left': '20px', 'width': '58%', 'display': 'inline-block', 'vertical-align': 'top'}),
    ])


# Define callback for filtered table
@app.callback(
    dash.dependencies.Output('filtered-table-container', 'children'),
    [dash.dependencies.Input('location-dropdown', 'value'),
     dash.dependencies.Input('warehouse-dropdown', 'value'),
     dash.dependencies.Input('van-type-dropdown', 'value'),
     dash.dependencies.Input('start-time-dropdown', 'value')]
)
def update_filtered_table(location, warehouse, van_type, start_time):
    filtered_df = df
    if location != 'All Locations':
        filtered_df = filtered_df[filtered_df['Loc'] == location]
    if warehouse != 'All Warehouses':
        filtered_df = filtered_df[filtered_df['warehouse'] == warehouse]
    if van_type != 'All Van Types':
        filtered_df = filtered_df[filtered_df['Vehicle'] == van_type]
    if start_time != 'All Start Locations':  # Update the variable name here
        filtered_df = filtered_df[filtered_df['planStartTime'] == start_time]

    # choose any colummns to display in the filtered_table section
    columns_to_display = ['Loc', 'warehouse', 'Vehicle', 'actStartTime', 'shipNo', 'Pred_Time']

    return html.Div(style={'overflowX': 'auto'},
                    children=[
                            dash_table.DataTable(
                            id='table-striped',
                            data = filtered_df[columns_to_display].to_dict('records'),
                            page_size=12,
                            style_header={
                                'font-family': 'Times New Romer',
                                'font-weight': 'bold',
                                'text-align': 'center'
                            },
                            style_data={
                                'font-family': 'Times New Romer',
                                'text-align': 'left'
                            }
                            )
                            ]
                    )

def draw_map(locs: str):
    if locs == None:
        #do nothing
        pass
    else:
        #standardize the input:
        ch = ['[', ']','\"','\'']
        for c in ch:
            locs = locs.replace(c,'')

        locs = locs.split(', ')
        loc_list = []
        # read the even number as longitude, odd as latitude
        for i in range(len(locs)):
            try: loc_list.append((locs[i],locs[i+1]))
            except IndexError:
                print('Location incomplete')
        print(loc_list)
        
        # use a new Map to overwrite the html file
        hongkong_map = folium.Map(location=[22.3193, 114.1694], zoom_start=11)
        for loc in loc_list:
            folium.Marker(location=loc).add_to(hongkong_map)

        new_map = hongkong_map
        new_map.save('map.html', 'w')


# Add the new Input for the active_cell property of the DataTable
@app.callback(
    [dash.dependencies.Output('shipment-details-container', 'children'),
    dash.dependencies.Output('map', 'srcDoc')],
    [dash.dependencies.Input('filtered-table-container', 'children'),
     dash.dependencies.Input('table-striped', 'active_cell')]
)
def update_shipment_details(filtered_table, active_cell):
    if active_cell is None:
        return html.Div(
        [
            html.Div('Shipment Details', style={'font-size': '16px', 'font-weight': 'bold', 'text-align': 'left', 'margin-left': '1%', 'margin-top': '1%'}),
            html.Div(
                [html.Div(style={'border': '1px solid black', 'height': '200px', 'width': '200px', 'display': 'inline-block', 'margin-right': '40px'}),
                 html.Div(style={'border': '1px solid black', 'height': '200px', 'width': '200px', 'display': 'inline-block', 'margin-right': '40px'}),
                 html.Div(style={'border': '1px solid black', 'height': '200px', 'width': '200px','display': 'inline-block', 'margin-right': '40px'}),
                 html.Div(style={'border': '1px solid black', 'height': '200px', 'width': '200px','display': 'inline-block'}),                ],
                style={'display': 'inline-block', 'margin-top': '10px', 'margin-left': '1%'}
            ),
        ],
        style={
            'border': '1px solid black',
            'height': '250px',
            'text-align': 'center',
        },
    )

    # Get the selected row index from the active_cell
    selected_row_idx = active_cell['row']

    # Retrieve the selected row data from the filtered table
    filtered_df = pd.DataFrame(filtered_table['props']['children'][0]['props']['data'])
    selected_row = filtered_df.iloc[selected_row_idx]

    # find the row in original dataframe with the selected shipNo(Suppose it is unique for any order)
    cur_ship_no = selected_row.shipNo
    selected_row = df[df['shipNo'].isin([cur_ship_no])]

    # Define the column names to display in the four square boxes
    columns_to_display = ['Vehicle', 'actStartTime', 'Distances', 'Pred_Time']
    
    # Call the helper function to update the current map file
    shortestRouteMap, shortestRouteLength, shortestRouteTime = VRP(graph, decode_route_list(selected_row["Route_Order"].item()))
    shortestRouteMap.save('map.html', 'w')

    # Create the four square boxes with the selected row's column values
    square_boxes = [
        html.Div(
            children=[
                html.P(column, style={'font-weight': 'bold'}),
                html.P(str(selected_row[column].item()))
            ],
            style={'border': '1px solid black', 'height': '200px', 'width': '200px', 'display': 'inline-block', 'margin-right': '40px'}
        ) for column in columns_to_display
    ]

    return html.Div(
        [
            html.Div('Shipment Details', style={'font-size': '16px', 'font-weight': 'bold', 'text-align': 'left', 'margin-left': '1%', 'margin-top': '1%'}),
            html.Div(
                square_boxes,
                style={'display': 'inline-block', 'margin-top': '10px', 'margin-left': '1%'}
            ),
        ],
        style={
            'border': '1px solid black',
            'height': '250px',
            'text-align': 'center',
        },
    ), open('map.html', 'r').read() # return two values, respectively the output specified in call_back function header

if __name__ == '__main__':
    app.run_server(debug=False)





Loading ./tomtom_speeds_weekday_rush_hours.json
Loading ./tomtom_speeds_weekday_non_rush_hours.json
Loading street network
Assigning speed data to edges
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/May/2023 01:04:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2023 01:04:42] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_7_0m1669048794.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2023 01:04:42] "GET /_dash-component-suites/dash/deps/react@16.v2_7_0m1669048794.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2023 01:04:42] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_7_0m1669048794.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2023 01:04:42] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_7_0m1669048794.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2023 01:04:42] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_7_0m1669048794.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2023 01:04:42] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_7_0m1669048794.js HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2023 01:04:42] "GET /_dash-component-s

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "c:\Users\tsang\anaconda3\lib\site-packages\dash\dash.py", line 1274, in dispatch
    ctx.run(
  Fil

127.0.0.1 - - [27/May/2023 01:04:43] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [27/May/2023 01:04:43] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2023 01:04:44] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2023 01:04:44] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2023 01:04:48] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "c:\Users\tsang\anaconda3\lib\site-packages\dash\dash.py", line 1274, in dispatch
    ctx.run(
  Fil

...

127.0.0.1 - - [27/May/2023 01:04:51] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [27/May/2023 01:04:55] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "c:\Users\tsang\anaconda3\lib\site-packages\dash\dash.py", line 1274, in dispatch
    ctx.run(
  Fil

127.0.0.1 - - [27/May/2023 01:05:08] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [27/May/2023 01:05:11] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\tsang\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "c:\Users\tsang\anaconda3\lib\site-packages\dash\dash.py", line 1274, in dispatch
    ctx.run(
  Fil

127.0.0.1 - - [27/May/2023 01:06:29] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [27/May/2023 01:06:32] "POST /_dash-update-component HTTP/1.1" 200 -
